In [1]:
#Import libraries
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import KFold, GridSearchCV
from catboost import Pool, CatBoostRegressor

/Users/orangethemeow/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#Read in the files
train = pd.read_csv("chocolate_train_for_cat.csv", keep_default_na=False)
test = pd.read_csv("chocolate_test_for_cat.csv", keep_default_na=False)

In [3]:
train.head()

,Unnamed: 0,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,...,spicy,sour,molasses,woody,vanilla_taste,sticky,fruit,smokey,company_continent,origin_continent
0,0,2454,5150,United States,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,...,0,0,0,0,0,0,1,0,NA,AF
1,1,2458,5150,United States,2019,Dominican republic,"Zorzal, batch 1",76.0,3.50,3,...,0,0,0,0,0,0,0,0,NA,NA
2,2,2454,5150,United States,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.25,3,...,0,0,0,0,0,0,0,0,NA,AF
3,3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,...,0,0,0,0,0,0,1,0,EU,SA
4,4,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,3.50,4,...,0,0,0,0,0,0,0,0,EU,SA


In [4]:
#Remove unnecessary columns from both training and test data
train.drop(columns=['Unnamed: 0', 'ref', 'company', 'company_location', 'country_of_bean_origin',
                  'specific_bean_origin_or_bar_name', 'taste'], inplace=True)

test.drop(columns=['Unnamed: 0', 'ref', 'company', 'company_location', 'country_of_bean_origin',
                  'specific_bean_origin_or_bar_name', 'taste'], inplace=True)

In [6]:
#Seperate target variable
y = train['rating']
X = train.drop(columns='rating')
y_test = test['rating']
X_test = test.drop(columns='rating')

In [7]:
#get column index, use 0, 2, 27, 28 as categorical features

cols = X.columns
[(c, X.columns.get_loc(c)) for c in cols if c in X]

[('review_date', 0),
 ('cocoa_percent', 1),
 ('counts_of_ingredients', 2),
 ('cocoa_butter', 3),
 ('vanilla', 4),
 ('lecithin', 5),
 ('salt', 6),
 ('sugar', 7),
 ('sweetener_without_sugar', 8),
 ('nutty', 9),
 ('sweet', 10),
 ('cocoa', 11),
 ('roasty', 12),
 ('creamy', 13),
 ('earthy', 14),
 ('sandy', 15),
 ('fatty', 16),
 ('floral', 17),
 ('intense', 18),
 ('spicy', 19),
 ('sour', 20),
 ('molasses', 21),
 ('woody', 22),
 ('vanilla_taste', 23),
 ('sticky', 24),
 ('fruit', 25),
 ('smokey', 26),
 ('company_continent', 27),
 ('origin_continent', 28)]

In [8]:
#Make sure that test set columns correpond to training set columns

test_cols = X.columns
[(c, X_test.columns.get_loc(c)) for c in test_cols if c in X]

[('review_date', 0),
 ('cocoa_percent', 1),
 ('counts_of_ingredients', 2),
 ('cocoa_butter', 3),
 ('vanilla', 4),
 ('lecithin', 5),
 ('salt', 6),
 ('sugar', 7),
 ('sweetener_without_sugar', 8),
 ('nutty', 9),
 ('sweet', 10),
 ('cocoa', 11),
 ('roasty', 12),
 ('creamy', 13),
 ('earthy', 14),
 ('sandy', 15),
 ('fatty', 16),
 ('floral', 17),
 ('intense', 18),
 ('spicy', 19),
 ('sour', 20),
 ('molasses', 21),
 ('woody', 22),
 ('vanilla_taste', 23),
 ('sticky', 24),
 ('fruit', 25),
 ('smokey', 26),
 ('company_continent', 27),
 ('origin_continent', 28)]

In [9]:
#10-fold cv
kf = KFold(n_splits=10, shuffle=True)

In [10]:
#Create model object
cat = CatBoostRegressor(iterations=500, loss_function='MAE', learning_rate=0.06)

In [11]:
#Run a 10-fold cross validation, record the MAE of each iteration

mae = []
for train_idx, val_idx in kf.split(X, y):
    
    train_pooled = Pool(X.loc[train_idx], y.loc[train_idx], cat_features=[0, 2, 27, 28])
    val_pooled = Pool(X.loc[val_idx], y.loc[val_idx], cat_features=[0, 2, 27, 28])
    
    cat.fit(train_pooled)
    preds = cat.predict(val_pooled)
 
    mae.append(metrics.mean_absolute_error(y[val_idx], preds))

0:	learn: 0.3483624	total: 55.8ms	remaining: 27.8s
1:	learn: 0.3460811	total: 63.3ms	remaining: 15.8s
2:	learn: 0.3445435	total: 75.5ms	remaining: 12.5s
3:	learn: 0.3429080	total: 82.9ms	remaining: 10.3s
4:	learn: 0.3413524	total: 88.2ms	remaining: 8.73s
5:	learn: 0.3399397	total: 104ms	remaining: 8.58s
6:	learn: 0.3394193	total: 106ms	remaining: 7.46s
7:	learn: 0.3371458	total: 118ms	remaining: 7.25s
8:	learn: 0.3355422	total: 124ms	remaining: 6.74s
9:	learn: 0.3339478	total: 131ms	remaining: 6.43s
10:	learn: 0.3322796	total: 138ms	remaining: 6.14s
11:	learn: 0.3309308	total: 145ms	remaining: 5.88s
12:	learn: 0.3299637	total: 150ms	remaining: 5.62s
13:	learn: 0.3284538	total: 157ms	remaining: 5.46s
14:	learn: 0.3272584	total: 162ms	remaining: 5.24s
15:	learn: 0.3261316	total: 167ms	remaining: 5.06s
16:	learn: 0.3248626	total: 172ms	remaining: 4.89s
17:	learn: 0.3235002	total: 177ms	remaining: 4.74s
18:	learn: 0.3223457	total: 182ms	remaining: 4.6s
19:	learn: 0.3211625	total: 186ms	rem

183:	learn: 0.2585307	total: 1.01s	remaining: 1.73s
184:	learn: 0.2584034	total: 1.01s	remaining: 1.73s
185:	learn: 0.2581240	total: 1.02s	remaining: 1.73s
186:	learn: 0.2578958	total: 1.06s	remaining: 1.78s
187:	learn: 0.2576421	total: 1.08s	remaining: 1.79s
188:	learn: 0.2573535	total: 1.16s	remaining: 1.91s
189:	learn: 0.2571030	total: 1.17s	remaining: 1.91s
190:	learn: 0.2568195	total: 1.19s	remaining: 1.93s
191:	learn: 0.2566403	total: 1.22s	remaining: 1.96s
192:	learn: 0.2563214	total: 1.25s	remaining: 1.99s
193:	learn: 0.2558830	total: 1.27s	remaining: 2.01s
194:	learn: 0.2558489	total: 1.28s	remaining: 2.01s
195:	learn: 0.2557215	total: 1.29s	remaining: 2s
196:	learn: 0.2555277	total: 1.3s	remaining: 2s
197:	learn: 0.2554145	total: 1.3s	remaining: 1.99s
198:	learn: 0.2551557	total: 1.31s	remaining: 1.98s
199:	learn: 0.2548453	total: 1.31s	remaining: 1.97s
200:	learn: 0.2545396	total: 1.32s	remaining: 1.97s
201:	learn: 0.2544099	total: 1.33s	remaining: 1.96s
202:	learn: 0.254138

379:	learn: 0.2270745	total: 2.15s	remaining: 679ms
380:	learn: 0.2268704	total: 2.16s	remaining: 673ms
381:	learn: 0.2266344	total: 2.16s	remaining: 668ms
382:	learn: 0.2265034	total: 2.17s	remaining: 663ms
383:	learn: 0.2264146	total: 2.17s	remaining: 656ms
384:	learn: 0.2262561	total: 2.18s	remaining: 650ms
385:	learn: 0.2261017	total: 2.18s	remaining: 644ms
386:	learn: 0.2260327	total: 2.18s	remaining: 638ms
387:	learn: 0.2259230	total: 2.19s	remaining: 631ms
388:	learn: 0.2258220	total: 2.19s	remaining: 625ms
389:	learn: 0.2256370	total: 2.2s	remaining: 619ms
390:	learn: 0.2255461	total: 2.2s	remaining: 614ms
391:	learn: 0.2253162	total: 2.21s	remaining: 608ms
392:	learn: 0.2251361	total: 2.21s	remaining: 602ms
393:	learn: 0.2249533	total: 2.21s	remaining: 595ms
394:	learn: 0.2248893	total: 2.22s	remaining: 589ms
395:	learn: 0.2246948	total: 2.22s	remaining: 583ms
396:	learn: 0.2245142	total: 2.22s	remaining: 577ms
397:	learn: 0.2244038	total: 2.23s	remaining: 571ms
398:	learn: 0.

42:	learn: 0.2965863	total: 205ms	remaining: 2.18s
43:	learn: 0.2958686	total: 210ms	remaining: 2.18s
44:	learn: 0.2951186	total: 214ms	remaining: 2.16s
45:	learn: 0.2947277	total: 221ms	remaining: 2.19s
46:	learn: 0.2942257	total: 225ms	remaining: 2.17s
47:	learn: 0.2935281	total: 230ms	remaining: 2.17s
48:	learn: 0.2931666	total: 235ms	remaining: 2.16s
49:	learn: 0.2925956	total: 239ms	remaining: 2.15s
50:	learn: 0.2916829	total: 243ms	remaining: 2.14s
51:	learn: 0.2910329	total: 250ms	remaining: 2.15s
52:	learn: 0.2908477	total: 253ms	remaining: 2.14s
53:	learn: 0.2905250	total: 257ms	remaining: 2.12s
54:	learn: 0.2904006	total: 261ms	remaining: 2.12s
55:	learn: 0.2896057	total: 266ms	remaining: 2.11s
56:	learn: 0.2890405	total: 271ms	remaining: 2.11s
57:	learn: 0.2887671	total: 277ms	remaining: 2.11s
58:	learn: 0.2883213	total: 284ms	remaining: 2.12s
59:	learn: 0.2879492	total: 291ms	remaining: 2.13s
60:	learn: 0.2873868	total: 295ms	remaining: 2.12s
61:	learn: 0.2870651	total: 298

240:	learn: 0.2400272	total: 1.13s	remaining: 1.22s
241:	learn: 0.2399238	total: 1.14s	remaining: 1.21s
242:	learn: 0.2397639	total: 1.15s	remaining: 1.22s
243:	learn: 0.2395851	total: 1.15s	remaining: 1.21s
244:	learn: 0.2393010	total: 1.16s	remaining: 1.21s
245:	learn: 0.2391132	total: 1.17s	remaining: 1.2s
246:	learn: 0.2390813	total: 1.17s	remaining: 1.2s
247:	learn: 0.2387853	total: 1.18s	remaining: 1.2s
248:	learn: 0.2385399	total: 1.18s	remaining: 1.19s
249:	learn: 0.2384469	total: 1.19s	remaining: 1.19s
250:	learn: 0.2383310	total: 1.19s	remaining: 1.18s
251:	learn: 0.2382071	total: 1.2s	remaining: 1.18s
252:	learn: 0.2380670	total: 1.2s	remaining: 1.17s
253:	learn: 0.2378365	total: 1.21s	remaining: 1.17s
254:	learn: 0.2376795	total: 1.21s	remaining: 1.16s
255:	learn: 0.2374660	total: 1.22s	remaining: 1.16s
256:	learn: 0.2371619	total: 1.22s	remaining: 1.16s
257:	learn: 0.2370659	total: 1.23s	remaining: 1.15s
258:	learn: 0.2368564	total: 1.23s	remaining: 1.15s
259:	learn: 0.236

...

229:	learn: 0.2452563	total: 1.2s	remaining: 1.4s
230:	learn: 0.2451522	total: 1.2s	remaining: 1.4s
231:	learn: 0.2448404	total: 1.22s	remaining: 1.4s
232:	learn: 0.2446779	total: 1.22s	remaining: 1.4s
233:	learn: 0.2444470	total: 1.23s	remaining: 1.4s
234:	learn: 0.2440409	total: 1.24s	remaining: 1.4s
235:	learn: 0.2438770	total: 1.24s	remaining: 1.39s
236:	learn: 0.2437930	total: 1.25s	remaining: 1.39s
237:	learn: 0.2436707	total: 1.25s	remaining: 1.38s
238:	learn: 0.2435640	total: 1.26s	remaining: 1.37s
239:	learn: 0.2434788	total: 1.26s	remaining: 1.37s
240:	learn: 0.2432991	total: 1.27s	remaining: 1.36s
241:	learn: 0.2431507	total: 1.27s	remaining: 1.36s
242:	learn: 0.2428025	total: 1.28s	remaining: 1.35s
243:	learn: 0.2424654	total: 1.28s	remaining: 1.35s
244:	learn: 0.2423173	total: 1.29s	remaining: 1.34s
245:	learn: 0.2421409	total: 1.29s	remaining: 1.34s
246:	learn: 0.2419430	total: 1.3s	remaining: 1.33s
247:	learn: 0.2415812	total: 1.3s	remaining: 1.32s
248:	learn: 0.2413812	

394:	learn: 0.2241684	total: 1.94s	remaining: 516ms
395:	learn: 0.2241028	total: 1.95s	remaining: 512ms
396:	learn: 0.2240377	total: 1.96s	remaining: 508ms
397:	learn: 0.2239757	total: 1.97s	remaining: 505ms
398:	learn: 0.2238280	total: 1.97s	remaining: 500ms
399:	learn: 0.2237684	total: 1.98s	remaining: 494ms
400:	learn: 0.2237148	total: 1.98s	remaining: 489ms
401:	learn: 0.2235917	total: 1.99s	remaining: 485ms
402:	learn: 0.2235235	total: 1.99s	remaining: 479ms
403:	learn: 0.2234691	total: 2s	remaining: 475ms
404:	learn: 0.2233738	total: 2s	remaining: 470ms
405:	learn: 0.2233155	total: 2s	remaining: 464ms
406:	learn: 0.2232393	total: 2.01s	remaining: 459ms
407:	learn: 0.2232046	total: 2.01s	remaining: 454ms
408:	learn: 0.2230672	total: 2.02s	remaining: 450ms
409:	learn: 0.2230277	total: 2.03s	remaining: 445ms
410:	learn: 0.2229873	total: 2.03s	remaining: 440ms
411:	learn: 0.2229633	total: 2.04s	remaining: 435ms
412:	learn: 0.2228756	total: 2.04s	remaining: 430ms
413:	learn: 0.2228030

73:	learn: 0.2855033	total: 395ms	remaining: 2.27s
74:	learn: 0.2850626	total: 402ms	remaining: 2.28s
75:	learn: 0.2846869	total: 411ms	remaining: 2.29s
76:	learn: 0.2842870	total: 419ms	remaining: 2.3s
77:	learn: 0.2838240	total: 426ms	remaining: 2.3s
78:	learn: 0.2836110	total: 432ms	remaining: 2.3s
79:	learn: 0.2833876	total: 436ms	remaining: 2.29s
80:	learn: 0.2832084	total: 441ms	remaining: 2.28s
81:	learn: 0.2832083	total: 441ms	remaining: 2.25s
82:	learn: 0.2830189	total: 448ms	remaining: 2.25s
83:	learn: 0.2825550	total: 452ms	remaining: 2.24s
84:	learn: 0.2822769	total: 456ms	remaining: 2.23s
85:	learn: 0.2818068	total: 465ms	remaining: 2.24s
86:	learn: 0.2816243	total: 471ms	remaining: 2.23s
87:	learn: 0.2816237	total: 472ms	remaining: 2.21s
88:	learn: 0.2812894	total: 479ms	remaining: 2.21s
89:	learn: 0.2810591	total: 483ms	remaining: 2.2s
90:	learn: 0.2805388	total: 488ms	remaining: 2.19s
91:	learn: 0.2800299	total: 495ms	remaining: 2.19s
92:	learn: 0.2797557	total: 500ms	r

270:	learn: 0.2384501	total: 1.32s	remaining: 1.12s
271:	learn: 0.2383822	total: 1.33s	remaining: 1.12s
272:	learn: 0.2383330	total: 1.34s	remaining: 1.11s
273:	learn: 0.2382802	total: 1.34s	remaining: 1.11s
274:	learn: 0.2381876	total: 1.35s	remaining: 1.1s
275:	learn: 0.2380187	total: 1.35s	remaining: 1.1s
276:	learn: 0.2379202	total: 1.36s	remaining: 1.09s
277:	learn: 0.2378427	total: 1.36s	remaining: 1.09s
278:	learn: 0.2377843	total: 1.36s	remaining: 1.08s
279:	learn: 0.2376450	total: 1.37s	remaining: 1.07s
280:	learn: 0.2375578	total: 1.37s	remaining: 1.07s
281:	learn: 0.2374677	total: 1.38s	remaining: 1.06s
282:	learn: 0.2373516	total: 1.38s	remaining: 1.06s
283:	learn: 0.2373094	total: 1.38s	remaining: 1.05s
284:	learn: 0.2371953	total: 1.39s	remaining: 1.05s
285:	learn: 0.2371483	total: 1.39s	remaining: 1.04s
286:	learn: 0.2371046	total: 1.39s	remaining: 1.03s
287:	learn: 0.2369514	total: 1.4s	remaining: 1.03s
288:	learn: 0.2367924	total: 1.4s	remaining: 1.02s
289:	learn: 0.23

469:	learn: 0.2193540	total: 2.27s	remaining: 145ms
470:	learn: 0.2193241	total: 2.28s	remaining: 140ms
471:	learn: 0.2192039	total: 2.29s	remaining: 136ms
472:	learn: 0.2191443	total: 2.29s	remaining: 131ms
473:	learn: 0.2191284	total: 2.29s	remaining: 126ms
474:	learn: 0.2190292	total: 2.3s	remaining: 121ms
475:	learn: 0.2189560	total: 2.3s	remaining: 116ms
476:	learn: 0.2187717	total: 2.31s	remaining: 111ms
477:	learn: 0.2187231	total: 2.31s	remaining: 106ms
478:	learn: 0.2186685	total: 2.32s	remaining: 102ms
479:	learn: 0.2185566	total: 2.32s	remaining: 96.8ms
480:	learn: 0.2183677	total: 2.33s	remaining: 91.9ms
481:	learn: 0.2183105	total: 2.33s	remaining: 87ms
482:	learn: 0.2182896	total: 2.33s	remaining: 82.1ms
483:	learn: 0.2182444	total: 2.34s	remaining: 77.3ms
484:	learn: 0.2181331	total: 2.34s	remaining: 72.4ms
485:	learn: 0.2180862	total: 2.35s	remaining: 67.6ms
486:	learn: 0.2180534	total: 2.35s	remaining: 62.8ms
487:	learn: 0.2180101	total: 2.36s	remaining: 58ms
488:	lear

In [12]:
#average validation mae
print(np.mean(mae))

0.30254573066061075


In [13]:
#Fit the model to test data
train_pooled = Pool(X, y, cat_features=[0, 2, 27, 28])
test_pooled = Pool(X_test, y_test, cat_features=[0, 2, 27, 28])

cat.fit(train_pooled)
y_pred = cat.predict(test_pooled)

0:	learn: 0.3461884	total: 9.08ms	remaining: 4.53s
1:	learn: 0.3436907	total: 51.3ms	remaining: 12.8s
2:	learn: 0.3421134	total: 100ms	remaining: 16.6s
3:	learn: 0.3404016	total: 107ms	remaining: 13.2s
4:	learn: 0.3385412	total: 112ms	remaining: 11.1s
5:	learn: 0.3363445	total: 117ms	remaining: 9.66s
6:	learn: 0.3345727	total: 123ms	remaining: 8.65s
7:	learn: 0.3334110	total: 128ms	remaining: 7.87s
8:	learn: 0.3320397	total: 134ms	remaining: 7.32s
9:	learn: 0.3311303	total: 139ms	remaining: 6.8s
10:	learn: 0.3291084	total: 144ms	remaining: 6.41s
11:	learn: 0.3271868	total: 150ms	remaining: 6.11s
12:	learn: 0.3256937	total: 155ms	remaining: 5.79s
13:	learn: 0.3244085	total: 159ms	remaining: 5.53s
14:	learn: 0.3228540	total: 164ms	remaining: 5.31s
15:	learn: 0.3211930	total: 174ms	remaining: 5.28s
16:	learn: 0.3207419	total: 177ms	remaining: 5.03s
17:	learn: 0.3194405	total: 182ms	remaining: 4.87s
18:	learn: 0.3182284	total: 188ms	remaining: 4.76s
19:	learn: 0.3170764	total: 193ms	remain

167:	learn: 0.2601028	total: 1.11s	remaining: 2.2s
168:	learn: 0.2598951	total: 1.13s	remaining: 2.21s
169:	learn: 0.2596061	total: 1.13s	remaining: 2.19s
170:	learn: 0.2593370	total: 1.13s	remaining: 2.18s
171:	learn: 0.2591946	total: 1.14s	remaining: 2.17s
172:	learn: 0.2589147	total: 1.14s	remaining: 2.16s
173:	learn: 0.2587560	total: 1.15s	remaining: 2.15s
174:	learn: 0.2584660	total: 1.15s	remaining: 2.14s
175:	learn: 0.2579721	total: 1.16s	remaining: 2.13s
176:	learn: 0.2575440	total: 1.16s	remaining: 2.12s
177:	learn: 0.2574247	total: 1.17s	remaining: 2.11s
178:	learn: 0.2571020	total: 1.17s	remaining: 2.1s
179:	learn: 0.2569686	total: 1.18s	remaining: 2.09s
180:	learn: 0.2568826	total: 1.18s	remaining: 2.08s
181:	learn: 0.2567302	total: 1.18s	remaining: 2.07s
182:	learn: 0.2565617	total: 1.19s	remaining: 2.06s
183:	learn: 0.2563913	total: 1.19s	remaining: 2.05s
184:	learn: 0.2562292	total: 1.2s	remaining: 2.04s
185:	learn: 0.2559929	total: 1.2s	remaining: 2.03s
186:	learn: 0.25

351:	learn: 0.2357476	total: 2.01s	remaining: 844ms
352:	learn: 0.2356857	total: 2.01s	remaining: 837ms
353:	learn: 0.2356474	total: 2.02s	remaining: 832ms
354:	learn: 0.2355571	total: 2.02s	remaining: 827ms
355:	learn: 0.2354161	total: 2.03s	remaining: 821ms
356:	learn: 0.2353126	total: 2.03s	remaining: 814ms
357:	learn: 0.2352095	total: 2.04s	remaining: 810ms
358:	learn: 0.2351455	total: 2.05s	remaining: 804ms
359:	learn: 0.2350936	total: 2.05s	remaining: 798ms
360:	learn: 0.2349756	total: 2.06s	remaining: 793ms
361:	learn: 0.2348527	total: 2.07s	remaining: 789ms
362:	learn: 0.2347169	total: 2.08s	remaining: 784ms
363:	learn: 0.2346381	total: 2.08s	remaining: 778ms
364:	learn: 0.2344897	total: 2.09s	remaining: 772ms
365:	learn: 0.2344222	total: 2.09s	remaining: 766ms
366:	learn: 0.2343491	total: 2.1s	remaining: 760ms
367:	learn: 0.2342947	total: 2.1s	remaining: 754ms
368:	learn: 0.2342664	total: 2.11s	remaining: 748ms
369:	learn: 0.2341317	total: 2.11s	remaining: 742ms
370:	learn: 0.

In [14]:
#sanity check, make sure the prediction values are in bound
print(np.min(y_pred), np.max(y_pred))

2.5800266570334873 3.6416762569646384


In [15]:
#test MAE
metrics.mean_absolute_error(y_test, y_pred)

0.30340975504365775